In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multioutput import MultiOutputRegressor
from sklearn.multioutput import MultiOutputClassifier

In [ ]:
X=[[1,2,5,3],[5,2,4,2],[2,3,5,3],[4,1,3,2],[1,3,4,2]]
y=[[2,3],[2],[0,1,3],[1,2,3],[0,1,2]]

In [ ]:
print(X)
print(y)

In [ ]:
y_trans=MultiLabelBinarizer().fit_transform(y)
y_trans

In [ ]:
type(X),type(y_trans)

In [ ]:
len(X),y_trans.shape

In [ ]:
fit_forest=RandomForestClassifier(n_estimators=100,random_state=1)

In [ ]:
model1=OneVsRestClassifier(fit_forest).fit(X,y_trans)
print(model1.predict([[2,3,2,4]]))
print(model1.predict(X))

In [ ]:
MultiOutputRegressor(fit_forest).fit(X,y_trans).predict([[2,3,2,4]])

In [ ]:
MultiOutputClassifier(linear_model.LogisticRegression(C=1e5)).fit(X, y_trans).predict([[2,3,2,4]])

In [ ]:
MultiOutputClassifier(fit_forest).fit(X, y_trans).predict([[2,3,2,4]])

In [ ]:
X_df=pd.DataFrame(X)
y_df=pd.DataFrame(y_trans)

In [ ]:
MultiOutputClassifier(fit_forest).fit(X_df, y_df).predict([[2,3,2,4]])

In [ ]:
yhat=model1.predict(X_df)
yhat

In [ ]:
phat=model1.predict_proba(X_df)
phat

In [ ]:
hat={}
for i in range(len(phat)):
    b={j:phat[i][j] for j in range(len(phat[i]))}
    b_sort=sorted(b.items(),key=lambda x:x[1],reverse=True)
    a=b_sort[0:2]
    c=''
    for j in a:
        temp=" ".join(map(str,j))
        c=c+temp+' '
    hat['id'+str(i)]=c

In [ ]:
hat

In [ ]:
final_stacking=pd.DataFrame(pd.Series(hat),index=hat.keys(),columns=['LabelConfidencePair'])
final_stacking.index.name='VideoId'

In [ ]:
final_stacking

In [ ]:
video_lvl_record = "/Users/shujiaohuang_Mac/Downloads/test-1.tfrecord"
vid_ids = []
labels = []
mean_rgb = []
mean_audio = []

for example in tf.python_io.tf_record_iterator(video_lvl_record):
    tf_example = tf.train.Example.FromString(example)
    vid_ids.append(tf_example.features.feature['video_id'].bytes_list.value[0].decode(encoding='UTF-8'))
    labels.append(tf_example.features.feature['labels'].int64_list.value)
    mean_rgb.append(tf_example.features.feature['mean_rgb'].float_list.value)
    mean_audio.append(tf_example.features.feature['mean_audio'].float_list.value)
tf_example

# Stacking

In [1]:
import numpy as np # linear algebra
import pandas as pd 
from sklearn.datasets import make_classification
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils import shuffle
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn import linear_model
import tensorflow as tf

# Write prediction to matrix

In [2]:
def stacking_feature(file):
    from sklearn.feature_extraction import DictVectorizer
    lines=open(file).readlines()
    result=[]
    file_id=[]
    for e,line in enumerate(lines):
        if e>0:
            row=line.strip().split(",")
            string_list=row[1].split(" ")
            float_list=[float(string_list[x]) if x%2==1 else string_list[x] for x in range(0,len(string_list))]
            newlist=[float_list[x:x+2] for x in range(0,len(float_list),2)]
            score_dict={}
            for x in range(0,len(newlist)):
                score_dict[newlist[x][0]]=(newlist[x][1])
            result.append(score_dict)
            file_id.append(row[0])
    feature=DictVectorizer(sparse=False).fit_transform(result)
    return file_id,feature

In [3]:
def _int64list_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=value))

def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _floats_feature(value):
    return tf.train.Feature(float_list=tf.train.FloatList(value=value)) 


class NumpyToTFHelper(object):
    def __init__(self,filenames):
        self.filenames = filenames
        self.current_file = -1
        self.current_index = 0 
        self.current_array = None
    def get_next(self):
        if self.current_file<len(self.filenames)-1:
            self.current_file += 1
            self.current_array = self.filenames[self.current_file]
        else:
            return None
        self.current_index+=1
        cur = self.filenames[self.current_index -1] 
        return np.matrix(cur).A1


#class NumpyToTFHelper(object):
#    def __init__(self,filenames):
#        self.filenames = filenames
#        self.current_file = -1
#        self.current_index = 0 
#        self.current_array = None
#    def get_next(self):
#        if self.current_file == -1: 
#            self.current_file += 1
#            self.current_array = np.load(self.filenames[self.current_file])['arr_0']
#        elif self.current_index == self.current_array.shape[0]-1:
#            if self.current_file == len(self.filenames) - 1:
#                return None
#            self.current_file += 1
#            self.current_array = np.load(self.filenames[self.current_file])['arr_0']
#            self.current_index = 0 
#        self.current_index += 1 
#        # return single 16000 length record
#        cur = self.current_array[self.current_index -1] 
#        #print(cur.shape)
#        #print(type(cur))
#        return np.matrix(cur).A1


## labels

In [4]:
val_y = pd.read_csv("/Users/shujiaohuang_Mac/Downloads/validate_labels.csv",header=None).as_matrix()

In [5]:
type(val_y)

numpy.ndarray

In [6]:
val_y.shape

(1401828, 2)

In [7]:
val_y

array([['--h7o0bAOpk', '0 2 581 111 55 317'],
       ['--O5HgNtvSc', '208 1 3073 1294 1073'],
       ['--q4Vg_nqEw', '0 656 2 19 2677'],
       ..., 
       ['zz9LRC0P8Os', '0 25 2 23'],
       ['zz4DceTfj-c', '1946 15'],
       ['zzvd6yEiOmk', '0 1 2 388 50']], dtype=object)

In [8]:
#val_y_aslist=[]
#for i in val_y[1]:
#    val_y_aslist.append(i.split(" "))

In [9]:
#print(type(val_y_aslist))
#print(len(val_y_aslist))
#print(val_y_aslist[0:2])

In [10]:
#val_y_bin=MultiLabelBinarizer().fit_transform(val_y_aslist)

In [11]:
#print(type(val_y_bin))
#print(val_y_bin.shape)

In [12]:
#val_y_bin

In [13]:
#val_y_bin[0:3]

In [14]:
#val_y_bin[0:3][0]

In [15]:
#np.savez('val_y_bin.npz',val_y_bin)

In [16]:
#y_train=pd.DataFrame(val_y_bin)
#y_train=val_y_bin

In [17]:
#print(type(y_train))
#print(y_train.shape)

In [18]:
#y_train[0:10],y_train.shape

In [19]:
#y_train[0:1000].shape

## X_train

In [ ]:
# merge tfrecords

In [ ]:
# read test.tfrecord
moe_val_1='/Users/shujiaohuang_Mac/Downloads/moe_val_5w_1.tfrecord'
for example in tf.python_io.tf_record_iterator(video_lvl_record):
    tf_example = tf.train.Example.FromString(example)
    break
tf_example

In [20]:
'''
file='/Users/shujiaohuang_Mac/Downloads/moe_2feature_eval_predictions.csv'
from sklearn.feature_extraction import DictVectorizer
lines=open(file).readlines()
result=[]
file_id=[]
for e,line in enumerate(lines):
    if e>0:
        if e<11:
            row=line.strip().split(",")
            string_list=row[1].split(" ")
            float_list=[float(string_list[x]) if x%2==1 else string_list[x] for x in range(0,len(string_list))]
            newlist=[float_list[x:x+2] for x in range(0,len(float_list),2)]
            score_dict={}
            for x in range(0,len(newlist)):
                score_dict[newlist[x][0]]=(newlist[x][1])
            result.append(score_dict)
            file_id.append(row[0])

moe_2feature_eval_stacking_new_feature=DictVectorizer(sparse=False).fit_transform(result)
'''

'\nfile=\'/Users/shujiaohuang_Mac/Downloads/moe_2feature_eval_predictions.csv\'\nfrom sklearn.feature_extraction import DictVectorizer\nlines=open(file).readlines()\nresult=[]\nfile_id=[]\nfor e,line in enumerate(lines):\n    if e>0:\n        if e<11:\n            row=line.strip().split(",")\n            string_list=row[1].split(" ")\n            float_list=[float(string_list[x]) if x%2==1 else string_list[x] for x in range(0,len(string_list))]\n            newlist=[float_list[x:x+2] for x in range(0,len(float_list),2)]\n            score_dict={}\n            for x in range(0,len(newlist)):\n                score_dict[newlist[x][0]]=(newlist[x][1])\n            result.append(score_dict)\n            file_id.append(row[0])\n\nmoe_2feature_eval_stacking_new_feature=DictVectorizer(sparse=False).fit_transform(result)\n'

In [8]:
moe_2feature_eval_stacking_new_feature_withid=stacking_feature('/Users/shujiaohuang_Mac/Downloads/moe_2feature_eval_predictions.csv')

In [ ]:
type(moe_2feature_eval_stacking_new_feature_withid),type(logistic_2feature_eval_stacking_new_feature_withid)

In [24]:
moe_2feature_eval_stacking_new_feature=moe_2feature_eval_stacking_new_feature_withid[1]
moe_2feature_eval_stacking_new_feature

array([[ 0.682859,  0.044098,  0.      , ...,  0.      ,  0.      ,  0.      ],
       [ 0.      ,  0.978585,  0.      , ...,  0.      ,  0.      ,  0.      ],
       [ 0.999238,  0.      ,  0.      , ...,  0.      ,  0.      ,  0.      ],
       ..., 
       [ 0.999989,  0.      ,  0.      , ...,  0.      ,  0.      ,  0.      ],
       [ 0.01214 ,  0.      ,  0.      , ...,  0.      ,  0.      ,  0.      ],
       [ 0.654479,  0.982899,  0.      , ...,  0.      ,  0.      ,  0.      ]])

In [25]:
moe_2feature_eval_stacking_new_feature.shape

(1401828, 4716)

In [26]:
# try
iter=1
num=4
X_top=moe_2feature_eval_stacking_new_feature[int(iter-1)*num:int(iter)*num]
y_top=val_y[int(iter-1)*num:int(iter)*num]
N_X = NumpyToTFHelper(X_top) 
n_X = N_X.get_next()
N_y = NumpyToTFHelper(y_top)
n_y = N_y.get_next()
while n_X is not None:
    example = tf.train.Example(
        features=tf.train.Features(
            feature={
                'video_id':_bytes_feature(bytes(n_y[0],'utf-8')),
                'labels':_int64list_feature([int(i) for i in n_y[1].split(' ')]),
                'New_feature':_floats_feature([float(i) for i in n_X])
            }
        )
    ) 
    print(example.features.feature['video_id'])
    n_X=N_X.get_next()
    n_y = N_y.get_next()

bytes_list {
  value: "--h7o0bAOpk"
}

bytes_list {
  value: "--O5HgNtvSc"
}

bytes_list {
  value: "--q4Vg_nqEw"
}

bytes_list {
  value: "--wpV2HK8zQ"
}



In [27]:
y_top

array([['--h7o0bAOpk', '0 2 581 111 55 317'],
       ['--O5HgNtvSc', '208 1 3073 1294 1073'],
       ['--q4Vg_nqEw', '0 656 2 19 2677'],
       ['--wpV2HK8zQ', '676 28']], dtype=object)

In [28]:
num=1401828
print(moe_2feature_eval_stacking_new_feature.shape[0]/num)
print([i for i in range(1,2)])
iter=1
print(iter<moe_2feature_eval_stacking_new_feature.shape[0]/num)
print(moe_2feature_eval_stacking_new_feature[int(iter-1)*num::].shape)

1.0
[1]
False
(1401828, 4716)


In [30]:
1401828/100000

14.01828

In [31]:
num=100000
for iter in range(9,16):
    if iter<moe_2feature_eval_stacking_new_feature.shape[0]/num:
        X_top=moe_2feature_eval_stacking_new_feature[int(iter-1)*num:int(iter)*num]
        y_top=val_y[int(iter-1)*num:int(iter)*num]
    else:
        X_top=moe_2feature_eval_stacking_new_feature[int(iter-1)*num::]
        y_top=val_y[int(iter-1)*num::]
    filename='moe_val_10w_'+str(iter)+'.tfrecord'
    writer = tf.python_io.TFRecordWriter(filename)
    N_X = NumpyToTFHelper(X_top) 
    n_X = N_X.get_next()
    N_y = NumpyToTFHelper(y_top)
    n_y = N_y.get_next()
    while n_X is not None:
        example = tf.train.Example(
            features=tf.train.Features(
                feature={
                    'video_id':_bytes_feature(bytes(n_y[0],'utf-8')),
                    'labels':_int64list_feature([int(i) for i in n_y[1].split(' ')]),
                    'New_feature':_floats_feature([float(i) for i in n_X])
                }
            )
        ) 
        writer.write(example.SerializeToString())
        n_X=N_X.get_next()
        n_y = N_y.get_next()
    writer.close()

In [4]:
logistic_2feature_eval_stacking_new_feature_withid=stacking_feature('/Users/shujiaohuang_Mac/Downloads/logistic_val_predictions.csv')

In [24]:
logistic_2feature_eval_stacking_new_feature=logistic_2feature_eval_stacking_new_feature_withid[1]

In [135]:
logistic_2feature_eval_stacking_new_feature

array([[ 0.907931,  0.02949 ,  0.      , ...,  0.      ,  0.      ,  0.      ],
       [ 0.      ,  0.976476,  0.      , ...,  0.      ,  0.      ,  0.      ],
       [ 0.999979,  0.      ,  0.      , ...,  0.      ,  0.      ,  0.      ],
       ..., 
       [ 0.999975,  0.016853,  0.      , ...,  0.      ,  0.      ,  0.      ],
       [ 0.048551,  0.004482,  0.      , ...,  0.      ,  0.      ,  0.      ],
       [ 0.465478,  0.954355,  0.      , ...,  0.      ,  0.      ,  0.      ]])

In [144]:
# Both moe and logisitc val tfrecord
for iteration in range(3,16):
    print(iteration)
    file_iter='10w_'+str(iteration)
    num=100000
    #file_iter='10w_'+str(iteration)
    moe_val='/Users/shujiaohuang_Mac/Downloads/moe_val_'+str(file_iter)+'.tfrecord'
    filename='both_moe_logistic_val_'+str(file_iter)+'.tfrecord'
    writer = tf.python_io.TFRecordWriter(filename)
    if iter<logistic_2feature_eval_stacking_new_feature.shape[0]/num:
        logistic_feature_temp=logistic_2feature_eval_stacking_new_feature[int(iteration-1)*num:int(iteration)*num]
    else:
        logistic_feature_temp=logistic_2feature_eval_stacking_new_feature[int(iteration-1)*num::]
    N_logistic=NumpyToTFHelper(logistic_feature_temp) 
    n_logistic = N_logistic.get_next()
    for example in tf.python_io.tf_record_iterator(moe_val):
        moe_example=tf.train.Example.FromString(example)
        example_both_moe_logistic=tf.train.Example(
            features=tf.train.Features(
                feature={
                    'video_id':moe_example.features.feature['video_id'],
                    'labels':moe_example.features.feature['labels'],
                    'moe_newfeature':moe_example.features.feature['New_feature'],
                    'logistic_newfeature':_floats_feature([float(i) for i in n_logistic])
                }
            )
        )
        writer.write(example_both_moe_logistic.SerializeToString())
        n_logistic=N_logistic.get_next()
    writer.close()

3
4
5
6
7
8
9
10
11
12
13
14
15


In [140]:
print(logistic_2feature_eval_stacking_new_feature_withid[0][200000])
print(logistic_2feature_eval_stacking_new_feature_withid[0][300000])
print(logistic_2feature_eval_stacking_new_feature_withid[0][400000])
print(logistic_2feature_eval_stacking_new_feature_withid[0][50000])

84vZctjYpkY
CdYRJB8bMpQ
HDPqsjayHpA
1Gj6Syodboc


In [142]:
print(logistic_2feature_eval_stacking_new_feature_withid[1][200000])
print(logistic_2feature_eval_stacking_new_feature_withid[1][300000])
print(logistic_2feature_eval_stacking_new_feature_withid[1][400000])


[ 0.        0.999973  0.       ...,  0.        0.        0.      ]
[ 0.        0.999711  0.021052 ...,  0.        0.        0.      ]
[ 0.9528    0.016312  0.010455 ...,  0.        0.        0.      ]


In [82]:
i=logistic_2feature_eval_stacking_new_feature_withid[0].index('VzifVf2frGc')
i

723156

In [83]:
logistic_2feature_eval_stacking_new_feature_withid[0][i]

'VzifVf2frGc'

In [84]:
logistic_2feature_eval_stacking_new_feature_withid[1][i]

array([ 0.      ,  0.117789,  0.101857, ...,  0.      ,  0.      ,  0.      ])

## X_test

In [4]:
logistic_2feature_test_blending_new_feature_withid=stacking_feature('/Users/shujiaohuang_Mac/Downloads/logistic_test_predictions.csv')

In [5]:
logistic_2feature_test_blending_new_feature=logistic_2feature_test_blending_new_feature_withid[1]

In [6]:
logistic_2feature_test_blending_new_feature

array([[ 0.305126,  0.955644,  0.      , ...,  0.      ,  0.      ,  0.      ],
       [ 0.16744 ,  0.04067 ,  0.012538, ...,  0.      ,  0.      ,  0.      ],
       [ 0.      ,  0.004725,  0.      , ...,  0.      ,  0.      ,  0.      ],
       ..., 
       [ 0.      ,  0.009766,  0.013569, ...,  0.      ,  0.      ,  0.      ],
       [ 0.150349,  0.      ,  0.      , ...,  0.      ,  0.      ,  0.      ],
       [ 0.      ,  0.      ,  0.      , ...,  0.      ,  0.      ,  0.      ]])

In [21]:
iteration=1
num=50000
moe_test='/Users/shujiaohuang_Mac/Downloads/moe_test_all/moe_test_5w_'+str(iteration)+'.tfrecord'
print(moe_test)
print(logistic_2feature_test_blending_new_feature.shape[0]/num)
print(iteration<logistic_2feature_test_blending_new_feature.shape[0]/num)
logistic_feature_temp=logistic_2feature_test_blending_new_feature[int(iteration-1)*num:int(iteration)*num]

/Users/shujiaohuang_Mac/Downloads/moe_test_all/moe_test_5w_1.tfrecord
14.0128
True


TypeError: unsupported operand type(s) for -: 'builtin_function_or_method' and 'int'

In [32]:
# Both moe and logisitc test tfrecord
for iteration in range(1,16):
    print(iteration)
    num=50000
    moe_test='/Users/shujiaohuang_Mac/Downloads/moe_test_all/moe_test_5w_'+str(iteration)+'.tfrecord'
    filename='both_moe_logistic_test_5w_'+str(iteration)+'.tfrecord'
    writer = tf.python_io.TFRecordWriter(filename)
    if iteration<logistic_2feature_test_blending_new_feature.shape[0]/num:
        logistic_feature_temp=logistic_2feature_test_blending_new_feature[int(iteration-1)*num:int(iteration)*num]
    else:
        logistic_feature_temp=logistic_2feature_test_blending_new_feature[int(iteration-1)*num::]
    N_logistic=NumpyToTFHelper(logistic_feature_temp) 
    n_logistic = N_logistic.get_next()
    for example in tf.python_io.tf_record_iterator(moe_test):
        moe_example=tf.train.Example.FromString(example)
        example_both_moe_logistic=tf.train.Example(
            features=tf.train.Features(
                feature={
                    'video_id':moe_example.features.feature['video_id'],
                    'labels':moe_example.features.feature['labels'],
                    'moe_newfeature':moe_example.features.feature['New_feature'],
                    'logistic_newfeature':_floats_feature([float(i) for i in n_logistic])
                }
            )
        )
        writer.write(example_both_moe_logistic.SerializeToString())
        n_logistic=N_logistic.get_next()
    writer.close()

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15


In [30]:
print(logistic_2feature_test_blending_new_feature_withid[1][0])
print(logistic_2feature_test_blending_new_feature_withid[1][50000])
print(logistic_2feature_test_blending_new_feature_withid[1][100000])
print(logistic_2feature_test_blending_new_feature_withid[1][150000])

[ 0.305126  0.955644  0.       ...,  0.        0.        0.      ]
[ 0.012251  0.008676  0.       ...,  0.        0.        0.      ]
[ 0.        0.        0.984217 ...,  0.        0.        0.      ]
[ 0.032548  0.006364  0.024194 ...,  0.        0.        0.      ]


In [ ]:
# read test.tfrecord
video_lvl_record='/Users/shujiaohuang_Mac/Downloads/test-1.tfrecord'
for example in tf.python_io.tf_record_iterator(video_lvl_record):
    tf_example = tf.train.Example.FromString(example)
    break
tf_example

In [4]:
from sklearn.feature_extraction import DictVectorizer
file='/Users/shujiaohuang_Mac/Downloads/moe_2feature_predictions.csv'
lines=open(file).readlines()
result=[]
test_id=[]
for e,line in enumerate(lines):
    if e>0:
        row=line.strip().split(",")
        string_list=row[1].split(" ")
        float_list=[float(string_list[x]) if x%2==1 else string_list[x] for x in range(0,len(string_list))]
        newlist=[float_list[x:x+2] for x in range(0,len(float_list),2)]
        score_dict={}
        for x in range(0,len(newlist)):
            score_dict[newlist[x][0]]=(newlist[x][1])
        result.append(score_dict)
        test_id.append(row[0])


In [5]:
feature=DictVectorizer(sparse=False).fit_transform(result)
feature.shape

(700640, 4716)

In [6]:
type(feature)

numpy.ndarray

In [7]:
type(test_id),len(test_id)

(list, 700640)

In [ ]:
# try
# without npz file
iter=1
num=3
test_id_temp=np.array([test_id[int(iter-1)*num:int(iter)*num]])
feature_temp=feature[int(iter-1)*num:int(iter)*num]
#test_moe_temp=np.concatenate((test_id_temp.T,feature_temp),axis=1)
#filename='moe_test_'+str(iter)+'_without_npz.tfrecord'
#writer = tf.python_io.TFRecordWriter(filename)
#N = NumpyToTFHelper(test_moe_temp) 
#n = N.get_next()
N_feature=NumpyToTFHelper(feature_temp)
n_feature=N_feature.get_next()
N_id=NumpyToTFHelper(test_id_temp.T)
n_id=N_id.get_next()
while n is not None:
    raw=n.tostring()
    example = tf.train.Example(
        features=tf.train.Features(
            feature={
                'video_id':_bytes_feature(bytes(n_id[0],'utf-8')),
                'labels':_int64list_feature([]),
                'New_feature':_floats_feature([float(i) for i in n_feature])
            }
        )
    ) 
    print(example.features.feature['video_id'])
    #writer.write(example.SerializeToString())
    n_feature=N_feature.get_next()
    n_id=N_id.get_next()
#writer.close()


In [44]:
n_id,n_feature

(None, None)

In [ ]:
print(test_id[int(iter-1)*num:int(iter)*num])

In [9]:
feature.shape[0]/50000

14.0128

In [52]:
# without npz file
num=50000
for iter in range(1,16):
    if iter<feature.shape[0]/num:
        test_id_temp=np.array([test_id[int(iter-1)*num:int(iter)*num]])
        feature_temp=feature[int(iter-1)*num:int(iter)*num]
    else:
        test_id_temp=np.array([test_id[int(iter-1)*num::]])
        feature_temp=feature[int(iter-1)*num::]
    filename='moe_test_5w_'+str(iter)+'.tfrecord'
    writer = tf.python_io.TFRecordWriter(filename)
    N_feature = NumpyToTFHelper(feature_temp) 
    n_feature = N_feature.get_next()
    N_id=NumpyToTFHelper(test_id_temp.T) 
    n_id = N_id.get_next()
    while n_id is not None:
        raw=n.tostring()
        example = tf.train.Example(
            features=tf.train.Features(
                feature={
                    'video_id':_bytes_feature(bytes(n_id[0],'utf-8')),
                    'labels':_int64list_feature([]),
                    'New_feature':_floats_feature([float(i) for i in n_feature])
                }
            )
        ) 
        writer.write(example.SerializeToString())
        n_feature=N_feature.get_next()
        n_id=N_id.get_next()
    writer.close()

In [7]:
moe_test_temp1='moe_test_2.tfrecord'
for example in tf.python_io.tf_record_iterator(moe_test_temp1):
    tf_example = tf.train.Example.FromString(example)

In [8]:
tf_example.features.feature['video_id']

bytes_list {
  value: "100646753"
}

In [9]:
type(test_id)

list

In [10]:
test_id.index('100646753')

19999

## stacking

In [ ]:
type(X_train),type(y_train)

In [ ]:
fit_log=linear_model.LogisticRegression(C=1e5).fit(X_train[0:10000],y_train[0][0:10000])

In [ ]:
blending_logistic_10000=MultiOutputClassifier(linear_model.LogisticRegression(C=1e5)).fit(X_train[0:10000], y_train[0:10000])


In [ ]:
phat_logistic=blending_logistic.predict_proba(X_test)

In [ ]:
hat={}
for i in range(len(phat)):
    b={j:phat[i][j] for j in range(len(phat[i]))}
    b_sort=sorted(b.items(),key=lambda x:x[1],reverse=True)
    a=b_sort[range(20)]
    c=''
    for j in a:
        temp=" ".join(map(str,j))
        c=c+temp+' '
    hat['id'+str(i)]=c
final_stacking=pd.DataFrame(pd.Series(hat),index=hat.keys(),columns=['LabelConfidencePair'])
final_stacking.index.name='VideoId'

In [ ]:
final_stacking.to_csv('stacking_logistic.csv')
